In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

from tqdm.notebook import trange, tqdm

### Train the agent
* Run it until he has a running average above the success_threshold
* Use a large number of episodes for the running average ( 1000+ ) so if even it falls into a privileged sample, it wont be prone to error

In [2]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
        ticker="IBM",
        use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    print('original threshold',env.success_threshold)
    
    env.success_threshold =0.25 # 25%
    
    print('current threshold',env.success_threshold)
    return env

# from src.environments.continuous.trading import environment

In [3]:
from src.agents.actor_critic.a2c import A2CAgent

In [4]:
agent=A2CAgent(environment, epochs=1, actor_learning_rate=0.000025,critic_learning_rate=0.000025,policy="CNN")
agent.load()



0.03489889871905054
[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/IbmStockTradingEnvironment-v0/2ec9feb00c05a2de76418e3bcb711337/models/a2c-actor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

[Errno 2] Unable to open file (unable to open file: name = 'storage/environments/IbmStockTradingEnvironment-v0/2ec9feb00c05a2de76418e3bcb711337/models/a2c-critic.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


0.03489889871905054


0.25

In [ ]:
agent.env.mode = "train"
agent.learning_log.episodes = 0
agent.learn(
    timesteps=-1, 
    log_every=100,
    success_threshold_lookback=1000,
    success_strict=True,
)

print('A2C + CNN + 0.25 threshold -> Executed in 1h 1m 4s, finished 18:43:51 2022-08-08')

Episode * 100 * Moving Avg Reward is ==> 0.00168 * Last Reward was ==> -0.01097
Episode * 200 * Moving Avg Reward is ==> 0.00540 * Last Reward was ==> 0.01132
Episode * 300 * Moving Avg Reward is ==> 0.00596 * Last Reward was ==> -0.01821
Episode * 400 * Moving Avg Reward is ==> 0.00924 * Last Reward was ==> 0.08276
Episode * 500 * Moving Avg Reward is ==> 0.01044 * Last Reward was ==> 0.01026
Episode * 600 * Moving Avg Reward is ==> 0.01196 * Last Reward was ==> 0.01050
Episode * 700 * Moving Avg Reward is ==> 0.01424 * Last Reward was ==> 0.03462
Episode * 800 * Moving Avg Reward is ==> 0.01755 * Last Reward was ==> 0.03666
Episode * 900 * Moving Avg Reward is ==> 0.02131 * Last Reward was ==> 0.03930
Episode * 1000 * Moving Avg Reward is ==> 0.02496 * Last Reward was ==> 0.06020
* Models saved *

New historical moving average record: 0.02505

* Models saved *

New historical moving average record: 0.02514

* Models saved *

New historical moving average record: 0.02514

* Models sav

* Models saved *

New historical moving average record: 0.03235

* Models saved *

New historical moving average record: 0.03246

* Models saved *

New historical moving average record: 0.03256

* Models saved *

New historical moving average record: 0.03264

* Models saved *

New historical moving average record: 0.03272

* Models saved *

New historical moving average record: 0.03278

* Models saved *

New historical moving average record: 0.03285

* Models saved *

New historical moving average record: 0.03293

* Models saved *

New historical moving average record: 0.03300

* Models saved *

New historical moving average record: 0.03301

* Models saved *

New historical moving average record: 0.03307

* Models saved *

New historical moving average record: 0.03322

* Models saved *

New historical moving average record: 0.03336

* Models saved *

New historical moving average record: 0.03337

* Models saved *

New historical moving average record: 0.03343

* Models saved *

New his

* Models saved *

New historical moving average record: 0.04200

* Models saved *

New historical moving average record: 0.04207

* Models saved *

New historical moving average record: 0.04212

* Models saved *

New historical moving average record: 0.04215

* Models saved *

New historical moving average record: 0.04219

* Models saved *

New historical moving average record: 0.04221

* Models saved *

New historical moving average record: 0.04226

* Models saved *

New historical moving average record: 0.04242

* Models saved *

New historical moving average record: 0.04245

* Models saved *

New historical moving average record: 0.04251

* Models saved *

New historical moving average record: 0.04260

* Models saved *

New historical moving average record: 0.04269

* Models saved *

New historical moving average record: 0.04283

* Models saved *

New historical moving average record: 0.04289

* Models saved *

New historical moving average record: 0.04292

* Models saved *

New his

* Models saved *

New historical moving average record: 0.05068

* Models saved *

New historical moving average record: 0.05073

Episode * 1400 * Moving Avg Reward is ==> 0.05069 * Last Reward was ==> 0.04650
* Models saved *

New historical moving average record: 0.05076

* Models saved *

New historical moving average record: 0.05081

* Models saved *

New historical moving average record: 0.05083

* Models saved *

New historical moving average record: 0.05090

* Models saved *

New historical moving average record: 0.05097

* Models saved *

New historical moving average record: 0.05110

* Models saved *

New historical moving average record: 0.05122

* Models saved *

New historical moving average record: 0.05130

* Models saved *

New historical moving average record: 0.05131

* Models saved *

New historical moving average record: 0.05133

* Models saved *

New historical moving average record: 0.05137

* Models saved *

New historical moving average record: 0.05153

* Models s

* Models saved *

New historical moving average record: 0.05914

* Models saved *

New historical moving average record: 0.05922

* Models saved *

New historical moving average record: 0.05932

* Models saved *

New historical moving average record: 0.05936

* Models saved *

New historical moving average record: 0.05941

* Models saved *

New historical moving average record: 0.05947

* Models saved *

New historical moving average record: 0.05948

* Models saved *

New historical moving average record: 0.05952

* Models saved *

New historical moving average record: 0.05962

* Models saved *

New historical moving average record: 0.05965

* Models saved *

New historical moving average record: 0.05967

* Models saved *

New historical moving average record: 0.05970

* Models saved *

New historical moving average record: 0.05981

* Models saved *

New historical moving average record: 0.05986

* Models saved *

New historical moving average record: 0.05996

* Models saved *

New his

...

* Models saved *

New historical moving average record: 0.14520

* Models saved *

New historical moving average record: 0.14524

* Models saved *

New historical moving average record: 0.14526

* Models saved *

New historical moving average record: 0.14531

* Models saved *

New historical moving average record: 0.14543

* Models saved *

New historical moving average record: 0.14543

* Models saved *

New historical moving average record: 0.14547

* Models saved *

New historical moving average record: 0.14550

* Models saved *

New historical moving average record: 0.14551

* Models saved *

New historical moving average record: 0.14558

* Models saved *

New historical moving average record: 0.14566

* Models saved *

New historical moving average record: 0.14577

* Models saved *

New historical moving average record: 0.14581

Episode * 4000 * Moving Avg Reward is ==> 0.14583 * Last Reward was ==> 0.14249
* Models saved *

New historical moving average record: 0.14583

* Models s

* Models saved *

New historical moving average record: 0.15222

* Models saved *

New historical moving average record: 0.15234

* Models saved *

New historical moving average record: 0.15242

* Models saved *

New historical moving average record: 0.15253

* Models saved *

New historical moving average record: 0.15255

* Models saved *

New historical moving average record: 0.15256

* Models saved *

New historical moving average record: 0.15258

* Models saved *

New historical moving average record: 0.15261

* Models saved *

New historical moving average record: 0.15270

* Models saved *

New historical moving average record: 0.15274

* Models saved *

New historical moving average record: 0.15280

* Models saved *

New historical moving average record: 0.15283

* Models saved *

New historical moving average record: 0.15284

* Models saved *

New historical moving average record: 0.15289

* Models saved *

New historical moving average record: 0.15292

* Models saved *

New his

* Models saved *

New historical moving average record: 0.15911

* Models saved *

New historical moving average record: 0.15921

* Models saved *

New historical moving average record: 0.15924

* Models saved *

New historical moving average record: 0.15933

* Models saved *

New historical moving average record: 0.15940

* Models saved *

New historical moving average record: 0.15943

* Models saved *

New historical moving average record: 0.15944

* Models saved *

New historical moving average record: 0.15945

* Models saved *

New historical moving average record: 0.15952

* Models saved *

New historical moving average record: 0.15954

* Models saved *

New historical moving average record: 0.15958

* Models saved *

New historical moving average record: 0.15967

* Models saved *

New historical moving average record: 0.15978

* Models saved *

New historical moving average record: 0.15988

* Models saved *

New historical moving average record: 0.15993

* Models saved *

New his

* Models saved *

New historical moving average record: 0.16569

* Models saved *

New historical moving average record: 0.16571

* Models saved *

New historical moving average record: 0.16573

* Models saved *

New historical moving average record: 0.16576

* Models saved *

New historical moving average record: 0.16579

* Models saved *

New historical moving average record: 0.16587

* Models saved *

New historical moving average record: 0.16591

* Models saved *

New historical moving average record: 0.16598

* Models saved *

New historical moving average record: 0.16604

* Models saved *

New historical moving average record: 0.16618

* Models saved *

New historical moving average record: 0.16620

Episode * 4700 * Moving Avg Reward is ==> 0.16618 * Last Reward was ==> 0.15688
* Models saved *

New historical moving average record: 0.16621

* Models saved *

New historical moving average record: 0.16622

* Models saved *

New historical moving average record: 0.16633

* Models s

* Models saved *

New historical moving average record: 0.17233

* Models saved *

New historical moving average record: 0.17234

* Models saved *

New historical moving average record: 0.17241

* Models saved *

New historical moving average record: 0.17251

* Models saved *

New historical moving average record: 0.17262

* Models saved *

New historical moving average record: 0.17271

* Models saved *

New historical moving average record: 0.17274

* Models saved *

New historical moving average record: 0.17279

* Models saved *

New historical moving average record: 0.17283

* Models saved *

New historical moving average record: 0.17285

* Models saved *

New historical moving average record: 0.17291

* Models saved *

New historical moving average record: 0.17303

* Models saved *

New historical moving average record: 0.17310



In [ ]:
agent.env.initial_investment

### Test the results
* Runs a set of episodes with **unseen data**
* Stores the results in a csv file for later consulting

In [ ]:
success = 0
n_tests = 10000

scores = []
targets = []
portfolio_target_ratios = []
initial_investments = []

for i in trange(n_tests):
    state = agent.env.reset(visualize=False,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    
    targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)
    initial_investments.append(agent.env.initial_investment)

    while not done:
        agent.env.render()
        
        # Select action
        action, action_onehot, prediction = agent.act(state)
        
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, info = agent.env.step(action)
        
        # End of episode, track scores here
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1

        step+=1
        state = next_state
        score += reward
    
    # Track scores and ratios
    scores.append(score)
    portfolio_target_ratios.append(info["portfolio_value"]/info["episode_target"] -1)
    
    
test_results_dataframe = pd.DataFrame([[
    n_tests,
    str(round(np.mean(scores)*100,3))+'%',
    str(round(np.mean(targets)*100,3))+'%',
    str(round(np.mean(portfolio_target_ratios)*100,3))+'%',
    str(round(min(scores)*100,3))+'%',
    str(round(max(scores)*100,3))+'%',
    str(round((success/n_tests)*100,3)) +'%'
]],
    columns=[
        '# Blind tests',
        '% Average portfolio return', 
        '% Desired portfolio return', 
        'Portfolio/Target rate',
        '% Historical minimum return',
        '% Historical maximum return', 
        '% Episodes concluded with positive outcome'
    ]
)


agent.results_writer.store_test_results(agent,test_results_dataframe)

test_results_dataframe.T

### Visual test
* Runs a set of episodes with **unseen data**
* See the evolution in real time


In [ ]:
success = 0
n_tests = 10

scores = []
targets = []


for i in trange(n_tests):
    state = agent.env.reset(visualize=True,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)
    initial_portfolio = agent.env.portfolio_value

    while not done:
        agent.env.render()
        
        # Select desired action
        action, action_onehot, prediction = agent.act(state)

        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)

        # End of episode, track scores here
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1

        step+=1
        state = next_state
        score += reward
    
    scores.append(score)

agent.env.close()

### Transactions profit
* Track how much the agent gained by doing purchases and sales
* Doesn't track the portfolio value 
* Already has the fees discounted

In [ ]:
success = 0
n_tests = 10000

test_results_data = []
test_results_columns = [
    'dataset id',
    'episode start date', 
    'episode end date',
    'episode working days',
    'total transactions',
    'total transaction profit',
    "dataframe hash",
    "agent hash"
]

print('NOTE: This test only tracks the transactions profit. ')

for i in trange(n_tests):
    trading_history = []
    state = agent.env.reset(visualize=False,mode="test")
    step = 0
    score = 0
    reward = 0
    done = False
    #targets.append((agent.env.episode_target-agent.env.initial_investment)/agent.env.initial_investment)
    #initial_portfolio = agent.env.portfolio_value
    
    j = 0
    num_stocks = 0
    while not done:
        agent.env.render()
        
        # Select desired action
        action, action_onehot, prediction = agent.act(state)
        current_price = agent.env.df.iloc[agent.env.current_step -1]['close']
        
        if action == agent.env.ACTIONS.BUY:
            discounted_price = current_price * (1+agent.env.fees.BUY) 
            if agent.env.stock_held < agent.env.maximum_stocks_held and agent.env.cash_in_hand >= discounted_price:
                num_stocks +=1
                trading_history.append(['buy', discounted_price])
        elif action == agent.env.ACTIONS.SELL:
            if agent.env.stock_held > 0:
                discounted_price = current_price * (1-agent.env.fees.SELL) 
                trading_history.append(['sell', discounted_price])
                num_stocks =0
                
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)

        # Update current state
        if done :
            if agent.env.portfolio_value > agent.env.initial_investment:
                success +=1
                
            profits = []
            purchases = []
            for e in trading_history:
                if e[0] == 'buy':
                    purchases.append(e[1])
                else:
                    profits.append((e[1]* len(purchases) - sum(purchases)) / sum(purchases))
                    purchases.clear() 
                    
            test_results_data.append([
                agent.env.dataset_idx,
                agent.env.df.iloc[agent.env.lookback].name,
                agent.env.df.iloc[-1].name,
                agent.env.window_size,
                len(profits),
                sum(profits)*100,
                agent.env.df_name,
                agent.hash
            ])


        step+=1
        state = next_state
        score += reward
    

agent.env.close()


pd.DataFrame(test_results_data,columns=test_results_columns)

In [ ]:
transaction_results_df = pd.DataFrame(test_results_data,columns=test_results_columns)

In [ ]:
transaction_results_df.sort_values(by="dataset id")

In [ ]:
transaction_results_df[transaction_results_df['total transaction profit'] <= 0]

In [ ]:
failed_episodes_df = transaction_results_df[transaction_results_df['total transaction profit'] <= 0]
failed_episodes_idx = failed_episodes_df['dataset id'].unique()
failed_episodes_idx.sort()

In [ ]:
len(transaction_results_df[transaction_results_df['total transaction profit'] <= 0]['dataset id'].unique()),len(transaction_results_df[transaction_results_df['total transaction profit'] > 0]['dataset id'].unique())

In [ ]:
import matplotlib.pyplot as plt 

for idx in failed_episodes_idx:
    plt.figure(figsize=(16,4))
    plt.plot(agent.env.df['close'])
    plt.title('#'+str(idx)+' - ' + str(len(failed_episodes_df[failed_episodes_df['dataset id'] == idx])) +'/' + str(len(transaction_results_df[transaction_results_df['dataset id'] == idx])))